In [235]:
# import the necessary package
import numpy as np
import pandas as pd
from   scipy.stats import norm
import scipy.optimize as opt
import math
from decimal import Decimal
from IPython.display import display
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
# Import the dataset
df = pd.read_stata('D:\\Temp\\mturk_clean_data_short.dta')

In [3]:
df

,buttonpresses,treatment,treatmentname
0,45,1.1,1c PieceRate
1,100,1.1,1c PieceRate
2,114,1.1,1c PieceRate
3,115,1.1,1c PieceRate
4,126,1.1,1c PieceRate
...,...,...,...
9856,3031,10,Gift Exchange
9857,3060,10,Gift Exchange
9858,3073,10,Gift Exchange
9859,3135,10,Gift Exchange


In [4]:
# Creating several varaiables in the dataset for estimation, same as original code did, but in a efficient way.
genre = ['payoff_per_100','payoff_charity_per_100',
         'dummy_charity','delay_wks','delay_dummy','gift_dummy','prob','weight_dummy'] 
for i in genre:
    if i != 'prob':
        df[F"{i}"] = 0
    else:
        df[F"{i}"] = 1

In [5]:
# Creating a dict for each treatment, and assigning the value in the dataset for each treatment. Detail is like following.
# Create piece-rate payoffs per 100 button presses (p)
# create payoff per 100 to charity and dummy charity (alpha/a) 
# create payoff per 100 delayed by 2 weeks and dummy delay
# probability weights to back out curvature and dummy
# dummy for gift exchange


assum = {'payoff_per_100':{'1.1':0.01,'1.2':0.1,'1.3':0.0,'2':0.001,'1.4':0.04,'4.1':0.01,'4.2':0.01,'6.2':0.02,'6.1':1},
 'payoff_charity_per_100':{'3.1':0.01,'3.2':0.1},
 'dummy_charity':{'3.1':1,'3.2':1},
 'delay_wks':{'4.1':2,'4.2':4},
 'delay_dummy':{'4.1':1,'4.2':1},
 'prob':{'6.2':0.5,'6.1':0.01},
 'weight_dummy':{'6.1':1},
 'gift_dummy':{'10':1}
 }

for task,payoff in assum.items():
    for key in payoff:
        df.loc[df.treatment == key, task] = payoff[key]

In [6]:
# Generating effort and log effort. authors round buttonpressed to nearest 100 value. If 0 set it to 25.

df['buttonpresses'] += 0.1 # python rounds 50 to 0, while stata to 100. by adding a small value we avoid this mismatch
df['buttonpresses_nearest_100'] = round(df['buttonpresses'],-2)
df.loc[df.buttonpresses_nearest_100 == 0, 'buttonpresses_nearest_100'] = 25
df['logbuttonpresses_nearest_100']  = np.log(df['buttonpresses_nearest_100'])

In [7]:
# Estimate procedure for s, k, gamma in benchmark case
# Define the benchmark sample by creating dummies equal to one if in treatment 1.1, 1.2, 1.3 
df['dummy1'] = (df['treatment'].isin(['1.1', '1.2','1.3'])).astype(int)# isin can be applied in pandas dataframe

In [8]:
# Set the initial values guess for the optimization procedure and scalers for k and s in the exp cost function case

st_values_exp = [0.015645717, 1.69443, 3.69198]#gamma_init_exp, k_init_exp, s_init_exp
st_values_power = [19.8117987, 1.66306e-10, 7.74996]#gamma_init_power, k_init_power, s_init_power
bp52_aut = [20.546,5.12e-70,3.17e-06]# The result of parameters of power cost function that author get.


In [196]:
def opt_param(effort, k_scaler, s_scaler, st_values,type):
    
    ''' effort: How many times the participants press the buttoms
        k_scaler: This is a tool to prvent from very large number showup in the calculation process, and it would not affect on the result. The k_scaler for
                   "exp" and "power" can be different.
        s_scaler:  This is a tool to prvent from very large number showup in the calculation process, and it would not affect on the result. The s_scaler for
                   "exp" and "power" can be different.
        st_values: This is the initial guess for the parameters.
        type: type can be "exp" or "power", to decide which cost function we are going to apply.
        '''
    
    def benchmark(pay100, g, k, s):
        
        ''' pay100: This is the payoff for basic assumption
            g,k,s: These are original parameters in both exp and power cost function. '''
        
        if type == "exp":
            check1 = k/k_scaler            # 'first'  component to compute f(x,θ). We call it check1 since it will enter a log, so we need to be careful with its value being > 0
            check2 = s/s_scaler + pay100   # 'second' component to compute f(x,θ)
        else:
            check1= max(k/k_scaler, 1e-115)                  # since check1 will enter log it must be greater than zero
            check2= np.maximum(s/s_scaler + pay100, 1e-10)   # np.maximum computes the max element wise. We do not want a negative value inside log    
              
        f_x = (-1/g * np.log(check1) +1/g * np.log(check2)) # f(x,θ) written above
        return f_x
    sol = opt.curve_fit(benchmark,
                    df.loc[df['dummy1']==1].payoff_per_100,
                    df.loc[df['dummy1']==1,effort],maxfev = 5000,
                    p0 = st_values)
    
    se = np.sqrt(np.diagonal(sol[1]))
    solo = [i/j for i,j in zip(sol[0],[1,k_scaler,s_scaler])]#Change it back to original scale
    se = [i/j for i,j in zip(se,[1,k_scaler,s_scaler])]#Change it back to original scale
    
    #Following code are for making nicer, comparable and understandable result
    sol_result = [0]*3
    se_result = [0]*3
    
    for i in range(0,len(sol[0])):
        if i == 0:
            sol_result[i] = round(solo[i],3)
            se_result[i] =  round(se[i],3)
        else:
            sol_result[i] = '{0:.2e}'.format(Decimal(solo[i]))
            se_result[i] = '{0:.2e}'.format(Decimal(se[i]))   
    
    if type == "exp":
        sse = np.round(((benchmark(df.loc[df['dummy1']==1].payoff_per_100,*sol[0])-df.loc[df['dummy1']==1,effort])**2),3)
        return sol_result,se_result
    
    else:
        sse = np.round(np.sum((benchmark(df.loc[df['dummy1']==1].payoff_per_100,*sol[0])-df.loc[df['dummy1']==1,effort])**2),3)#Calculate the squared for our result
        bp52_aut = [20.546,5.12e-13,3.17]
        sse_aut = np.round(np.sum((benchmark(df.loc[df['dummy1']==1].payoff_per_100,*bp52_aut)-df.loc[df['dummy1']==1,effort])**2),3)#Calculate the squared for authors result
        return sol_result,se_result,sse,sse_aut
    

In [197]:
#The estimation result and standard error.
be54,se54= opt_param(effort = "buttonpresses_nearest_100",k_scaler = 1e+16, s_scaler= 1e+6,st_values = st_values_exp,type = "exp")
be54

[0.016, '1.71e-16', '3.72e-6']

In [11]:
#Making the author's result nicer, comparable and understandable.
aut = [0]*3
for i in range(0,len(bp52_aut)):
        if i == 0:
            aut[i] = round(bp52_aut[i],3)
        else:
            aut[i] = '{0:.2e}'.format(Decimal(bp52_aut[i]))
aut

[20.546, '5.12e-70', '3.17e-6']

In [198]:
#The estimation result and standard error.
bp52,sp52,sse,sse_aut = opt_param(effort = "logbuttonpresses_nearest_100",k_scaler = 1e+57, s_scaler= 1e+6,st_values = st_values_power,type = "power")
sse

670.61

In [199]:
sse_aut

672.387

In [14]:
#The previous function "opt_param" is the replication of the original code but in a more efficient way. 
#Here I try to change part of the assumption of the code.I rename the func as my_ver_opt 
#Trying to get closer to the result of the paper and also makes the code simpler, now there is no argument called "type"!
#But in order to keep the consistency of the replication, I would not used the result from this part.
 

In [15]:
def my_ver_opt(effort,st_values):
    '''
    effort: How many times the participants press the button
    
    st_values: This is the initial guess for the parameters.
    '''
    
    def my_benchmark(pay100, g, k, s):
        ''' 
        pay100: This is the payoff for basic assumption
            
        g,k,s: These are original parameters in both exp and power cost function. 
        '''

        if k <= 0 or (s+pay100).values.min() <= 0:#making sure that the value in log function would not be negative
            return 1e-16
        f_x = (-1/g * np.log(k) +1/g * np.log(s+pay100))   # f(x,θ) written above
        return f_x
    sol = opt.curve_fit(my_benchmark,
                        df.loc[df['dummy1']==1].payoff_per_100,
                        df.loc[df['dummy1']==1,effort],
                        p0 = st_values,maxfev = 5000)
    return sol[0],np.sqrt(np.diagonal(sol[1]))

In [16]:
my_modified = my_ver_opt("buttonpresses_nearest_100",st_values=st_values_exp)# This result is closer to NLS on individual effort on Exponential effort cost.

In [17]:
my_modified

(array([1.56410708e-02, 1.70926793e-16, 3.72225994e-06]),
 array([4.14866499e-03, 1.49076434e-15, 9.16241801e-06]))

In [18]:
my_modified_power = my_ver_opt("logbuttonpresses_nearest_100",st_values=st_values_power)
my_modified_power


(array([2.11535441e+01, 8.07100252e-72, 1.40406905e-06]),
 array([7.36831559e+00, 4.63109452e-70, 5.01334839e-06]))

In [19]:
#The original python code try to use a different package to find the NLS estimates to see if they get closer to the authors' but without success. 
#opt.least_squares takes as input directly the squared residuals, so we need to specify a different objective function. 

def other_opi(effort, k_scaler,s_scaler,type,opt_type,st_values):
    '''
    effort: How many times the participants press the button
    
    k_scaler: This is a tool to prvent from very large number showup in the calculation process, and it would not affect on the result. The k_scaler for
                   "exp" and "power" can be different.
    s_scaler:  This is a tool to prvent from very large number showup in the calculation process, and it would not affect on the result. The s_scaler for
                   "exp" and "power" can be different.
    st_values: This is the initial guess for the parameters.
    
    opt_type: There are two optimization method can be applied here,opt.minimize and opt.least_squares. So Here opt_type can be "mini" or "ls"
    
    type: type can be "exp" or "power", to decide which cost function we are going to apply.
    '''
    def benchmark_other(params):
        '''
        params: This include g,k,s
        '''
        pay100 = np.array(df.loc[df['dummy1']==1].payoff_per_100)
        buttonpresses = np.array(df.loc[df['dummy1']==1,effort])
        g, k, s = params
        if type == "exp":
            check1 = k/k_scaler            
            check2 = s/s_scaler + pay100
        elif type == "power":   
            check1= max(k/k_scaler, 1e-100)
            check2= np.maximum(s/s_scaler + pay100, 1e-10)   
        if opt_type == "ls":    
            f_x = (0.5*((-1/g * np.log(check1) +1/g * np.log(check2))-buttonpresses)**2)#Now the f_x is different from the one previous function.
        else:
            f_x = np.sum(0.5*((-1/g * np.log(check1) +1/g * np.log(check2))-buttonpresses)**2)# The way to calculate the f_x in these optimization method is different.
        return f_x
    sol_result = [0]*3
    if opt_type == "ls":#least squared method
        sol = opt.least_squares(benchmark_other,
                        x0 = st_values,
                        xtol=1e-15,
                        ftol=1e-15,
                        gtol=1e-15,
                        method='lm')
        
        #Following code are for making nicer, comparable and understandable result
        solo = [i/j for i,j in zip(sol.x,[1,k_scaler,s_scaler])]#change back to the original scale
        sse = np.round((2*benchmark_other(sol.x)).sum(),3)#calculate the sum of squared error
        for i in range(0,3):
            if i==0:
                sol_result[i] = round(solo[i],3)   
            else:
                sol_result[i] = '{0:.2e}'.format(Decimal(solo[i]))
        return sol_result,sse
    
    elif opt_type == "mini":
        sol = opt.minimize(benchmark_other,
                       x0 = st_values,
                       method='Nelder-Mead',
                       options={'maxiter': 2500})
        solo = [i/j for i,j in zip(sol.x,[1,k_scaler,s_scaler])]#change back to the original scale
        sse = np.round((2*benchmark_other(sol.x)),3)#calculate the sum of squared error
        for i in range(0,3):
            if i==0:
                sol_result[i] = round(solo[i],3)   
            else:
                sol_result[i] = '{0:.2e}'.format(Decimal(solo[i])) 
        return sol_result,sse 

In [20]:
#Following is the estimation result for different cost function and optimzation method.
bp52_least_squaree = other_opi("logbuttonpresses_nearest_100",k_scaler = 1e+57, s_scaler= 1e+6,type="power",opt_type="ls",st_values = st_values_power)
bp52_least_squaree

([21.787, '2.17e-69', '9.56e-8'], 1112.339)

In [21]:
bp52_optt = other_opi("logbuttonpresses_nearest_100",k_scaler = 1e+57, s_scaler= 1e+6,type = "power",opt_type="mini",st_values = st_values_power)
bp52_optt

([21.266, '3.45e-72', '1.33e-6'], 670.61)

In [22]:
be52_ls = other_opi("buttonpresses_nearest_100",k_scaler = 1e+16, s_scaler= 1e+6,type = "exp",opt_type="ls",st_values=st_values_exp)
be52_ls


([0.011, '8.22e-12', '1.16e-4'], 731058666.567)

In [23]:
be52_optt = other_opi("buttonpresses_nearest_100",k_scaler = 1e+16, s_scaler= 1e+6,type = "exp",opt_type="mini",st_values=st_values_exp)
be52_optt

([0.016, '1.71e-16', '3.72e-6'], 709519964.727)

In [24]:
bp52

[21.1939, '5.95e-72', '1.38e-6']

In [200]:

pn = ["Curvature γ of cost function","Level k of cost of effort", "Intrinsic motivation s","Min obj. function"]
r1 = pd.DataFrame({'parameters':pn,'curve_fit':[*bp52,sse],
                   'least_square':[*bp52_least_squaree[0],bp52_least_squaree[1]],
                   'minimize_nd':[*bp52_optt[0],bp52_optt[1]],
                   'authors':[*aut,sse_aut]})

In [201]:
display(r1)# Notice that the reusult of original code and my code have some difference, 
#which is because that the scale for the original code and the paper is different(the original code didn't change back the result from the scaler), 
#and here I try to replicate result of the paper, which can correspond to table 5 power cost function part.
# If don't change back the scale from scaler, the result is exactly the same as the one in original python code.

,parameters,curve_fit,least_square,minimize_nd,authors
0,Curvature γ of cost function,21.194,21.787,21.266,20.546
1,Level k of cost of effort,5.95e-72,2.17e-69,3.45e-72,5.12e-70
2,Intrinsic motivation s,1.38e-6,9.56e-8,1.33e-6,3.17e-6
3,Min obj. function,670.61,1112.339,670.61,672.387


In [27]:
my_modified

(array([1.56410708e-02, 1.70926793e-16, 3.72225994e-06]),
 array([4.14866499e-03, 1.49076434e-15, 9.16241801e-06]))

In [236]:
#Update,also adding the authors result for comparasion
#Here I also adding the outcome of exponential cost of effort
pn = ["Curvature γ of cost function","Level k of cost of effort", "Intrinsic motivation s"]
be52_aut = [0.0156,1.71E-16,3.72E-06]
r2 = pd.DataFrame({'parameters':pn,'curve_fit':[*be54],
                   'least_square':[*be52_ls[0]],
                   'minimize_nd':[*be52_optt[0]],
                   'authors':[round(be52_aut[0],3),('{0:.2e}'.format(Decimal(be52_aut[1]))),('{0:.2e}'.format(Decimal(be52_aut[2])))]})
display(r2)


,parameters,curve_fit,least_square,minimize_nd,authors
0,Curvature γ of cost function,0.016,0.011,0.016,0.016
1,Level k of cost of effort,1.71e-16,8.22e-12,1.71e-16,1.71e-16
2,Intrinsic motivation s,3.72e-6,1.16e-4,3.72e-6,3.72e-6


In [242]:
r2.iloc[0,1:5]

curve_fit       0.016
least_square    0.011
minimize_nd     0.016
authors         0.016
Name: 0, dtype: object

In [246]:
method = ['curve_fit','least_squared','minimize_nd','authors']

fig = make_subplots(rows = 2, cols= 2, shared_yaxes=False)
                    #subplot_titles=('Estimation of gamma', 'Estimation of Probability weighting'))

trace1 = go.Bar(
    x=method,
    y=r1.iloc[0,1:5],
    name='gamma estimation in power function',
    textposition='auto',
    text=r1.iloc[0,1:5]
)

trace2 = go.Bar(
    x=method,
    y=r2.iloc[0,1:5],
    name='my gamma estimation in exp function',
    textposition='auto',
    text=r2.iloc[0,1:5]
)

trace3 = go.Bar(
    x=method,
    y=r1.iloc[3,1:5],
    name='sum of squared error',
    textposition='auto',
    text=r1.iloc[3,1:5]
)

#fig.append_trace(trace3, 1,1)
fig.append_trace(trace1,1,1)
fig.append_trace(trace2,1,2)
fig.append_trace(trace3,2,1)

# Change the bar mode
fig.update_layout(barmode='group',
    title='Comparison of parameters and sse between my result and the result of authors')
fig.show()

In [30]:
#display(r2) 
#The estimation from the original code of parameter "k" and "s" seems a bit different from the authors. 
#I think it is result from the basic assumption of the model.
#I tried to modified it from curve fit. 

In [31]:
# Create dummies for this specification
# Define the initial guess for new parameters alpha, a, beta, delta, gift
df['samplenw'] = (df['treatment'].isin(['1.1','1.2','1.3','3.1','3.2','4.1','4.2','10'])).astype(int)
stvale_spec = [0.003, 0.13, 1.16, 0.75, 5e-6] #alpha_init, a_init, beta_init, delta_init, gift_init

In [32]:
# Define the f(x,θ) to estimate all parameters but the probability weight

def treatment_a(effort,k_scaler,s_scaler,type,st_values):
    '''
    effort: How many times the participants press the button
    
    k_scaler: This is a tool to prvent from very large number show up in the calculation process, and it would not affect on the result. The k_scaler for
                   "exp" and "power" can be different.
    s_scaler:  This is a tool to prvent from very large number show up in the calculation process, and it would not affect on the result. The s_scaler for
                   "exp" and "power" can be different.
    
    type: type can be "exp" or "power", to decide which cost function we are going to apply.
    
    st_values: This is the initial guess for the parameters k,gamma,s.

    '''
    def noweight(xdata, g, k, s, alpha, a, gift, beta, delta):
        '''
        xdata: This the vector containing the explanatory variables
           
        g, k, s: These are the same parameters from before
        
        alpha: This the pure altruism coefficient
          
        a: This is the warm glow coefficient
          
        gift: This is the gift exchange coefficient Δs

        beta: This is the present bias paramater

        delta: This is the (weekly) discount factor
          
        '''
        pay100 = xdata[0]
        gd = xdata[1] #gd is gift dummy
        dd = xdata[2] #dd is delay dummy
        dw = xdata[3] #dw is delay weeks
        paychar = xdata[4] #paychar is pay in charity treatment
        dc = xdata[5] #dc is dummy charity
        if type == "exp":
            check1 = k/k_scaler
            check2 = s/s_scaler + gift*0.4*gd + (beta**dd)*(delta**dw)*pay100 + alpha*paychar + a*0.01*dc
        elif type == "power":
            check1= max(k/k_scaler, 1e-115)
            check2= np.maximum(s/s_scaler + gift*0.4*gd + (beta**dd)*(delta**dw)*pay100 + alpha*paychar + a*0.01*dc, 1e-10)  
        f_x = (-1/g * np.log(check1) + 1/g*np.log(check2))
        return f_x

# Find the solution to the problem by non-linear least squares 
    
    st_valuesnoweight = np.concatenate((st_values,stvale_spec)) # starting values

    args = [df.loc[df['samplenw']==1].payoff_per_100, df.loc[df['samplenw']==1].gift_dummy, df.loc[df['samplenw']==1].delay_dummy,
            df.loc[df['samplenw']==1].delay_wks, df.loc[df['samplenw']==1].payoff_charity_per_100, df.loc[df['samplenw']==1].dummy_charity]

    sol = opt.curve_fit(noweight, 
                    args,
                    df.loc[df['samplenw']==1,effort],
                    st_valuesnoweight)
    
    sol_result = [0]*8
    se_result = [0]*8
    se = np.sqrt(np.diagonal(sol[1]))
    for i in range(0,len(sol[0])):
        if i==5:
            sol_result[i] = '{0:.2e}'.format(Decimal(sol[0][i]))
            se_result[i] = '{0:.2e}'.format(Decimal(se[i]))   
        else:
            sol_result[i] = round(sol[0][i],3)
            se_result[i] = round(se[i],3)
    return sol_result,se_result
    #if type == "power":
        #nwest_aut = [20.546, 5.12e-70, 3.17e-06, 0.0064462, 0.1818249, 0.0000204, 1.357934, 0.7494928]
        # = np.sum((noweight(args,*sol[0])-df.loc[df['samplenw']==1,effort])**2)
        #sse_aut = np.sum((noweight(args,*nwest_aut)-df.loc[df['samplenw']==1,effort])**2)這樣套進去數字會不對，因為scale不對
        #return sol[0],np.sqrt(np.diagonal(sol[1])),sse_our,sse_aut
        #return sol_result,se_result #,sse_our,sse_aut
    #elif type == "exp":
        #return sol[0],np.sqrt(np.diagonal(sol[1]))
        #return sol_result,se_result


In [33]:
#Following are the estimation result
be56,se56 = treatment_a("buttonpresses_nearest_100",k_scaler = 1e+16, s_scaler= 1e+6,type = "exp",st_values = st_values_exp)
se56

C:\Users\ua896\AppData\Local\Temp\ipykernel_50424\2706465427.py:46: RuntimeWarning: invalid value encountered in log
  f_x = (-1/g * np.log(check1) + 1/g*np.log(check2))


[0.004, 14.857, 9.132, 0.011, 0.143, '4.82e-5', 1.302, 0.239]

In [34]:
bp53,se53 = treatment_a("logbuttonpresses_nearest_100",k_scaler = 1e+57, s_scaler= 1e+6,type = "power",st_values = st_values_power)
bp53
#sse_our,sse_our,sse_aut 

[21.082, 0.0, 1.452, 0.013, 0.265, '3.25e-5', 1.616, 0.751]

In [35]:
se53[3:8]

[0.03, 0.288, '8.14e-5', 2.053, 0.292]

In [36]:
#Adding authors result
aut_power = [round(20.546,4),'{0:.2e}'.format(Decimal(5.12e-70)),'{0:.2e}'.format(Decimal(3.17e-06)),round(0.006,3),
             round(0.182,3),'{0:.2e}'.format(Decimal(2.04e-05)),round(1.36,2),round(0.75,2)]


aut_exp = [round(0.0156,4),'{0:.2e}'.format(Decimal(1.71e-16)),'{0:.2e}'.format(Decimal(3.72e-06)),round(0.07,3),round(0.035,3),
           '{0:.2e}'.format(Decimal(3e-05)),round(0.79,2),round(0.86,2)]

In [212]:
#換名字
# Create and save the dataframe for table 5 NLS estimates. 

params_name = ["Curvature γ of cost function", "Level k of cost of effort", "Intrinsic motivation s","Social preferences α",
                "Warm glow coefficient a","Gift exchange Δs", "Present bias β","(Weekly) discount factor δ"]

be5 = be54[0:3]+be56[3:8]
se5 = se54[0:3]+se56[3:8]
bp5 = bp52[0:3]+bp53[3:8]
sp5 = sp52[0:3]+se53[3:8]


t5 = pd.DataFrame({'parameters':params_name,'power_est':bp5,'power_aut':aut_power,'se_p':sp5,'exp_est':be5,'se_e':se5,'exp_aut':aut_exp})
print('Table 5: non-linear-least-squares estimates of behavioural parameters')
display(t5)

Table 5: non-linear-least-squares estimates of behavioural parameters


,parameters,power_est,power_aut,se_p,exp_est,se_e,exp_aut
0,Curvature γ of cost function,21.194,20.546,7.399,0.016,0.004,0.0156
1,Level k of cost of effort,5.95e-72,5.12e-70,3.33e-70,1.71e-16,1.49e-15,1.71e-16
2,Intrinsic motivation s,1.38e-6,3.17e-6,4.93e-6,3.72e-6,9.16e-6,3.72e-6
3,Social preferences α,0.013,0.006,0.03,0.004,0.011,0.07
4,Warm glow coefficient a,0.265,0.182,0.288,0.143,0.143,0.035
5,Gift exchange Δs,3.25e-5,2.04e-5,8.14e-5,2.35e-5,4.82e-5,3.00e-5
6,Present bias β,1.616,1.36,2.053,1.237,1.302,0.79
7,(Weekly) discount factor δ,0.751,0.75,0.292,0.754,0.239,0.86


In [219]:
t5.filter(regex= 'power_').iloc[2,:]

power_est    1.38e-6
power_aut    3.17e-6
Name: 2, dtype: object

In [232]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot_func0(a):
    fig = make_subplots(rows = 3, cols= 2, shared_yaxes=False)#subplot_titles=('Estimation of gamma', 'Estimation of Probability weighting')
                        
    trace1 = go.Bar(
        x=['my gamma estimation','authors estimation'],
        y=t5.filter(regex= a).iloc[0,:],
        name='gamma estimation comparison',
        textposition='auto',
        text=t5.filter(regex= a).iloc[0,:]
    )

    trace2 = go.Bar(
        x=['my Social preferences estimation','author estimation'],
        y=t5.filter(regex= a).iloc[3,:],
        name='Social preferences estimation comparison',
        textposition='auto',
        text=t5.filter(regex= a).iloc[3,:]
    )

    trace3 = go.Bar(
        x=['my coefficient a estimation','author estimation'],
        y=t5.filter(regex= a).iloc[4,:],
        name='Warm glow coefficient a estimation comparision',
        textposition='auto',
        text=t5.filter(regex= a).iloc[4,:]
    )

    trace4 = go.Bar(
        x=['my Present bias β estimation','author estimation'],
        y=t5.filter(regex= a).iloc[6,:],
        name='Present bias β estimation comparision',
        textposition='auto',
        text=t5.filter(regex= a).iloc[6,:]
    )

    trace5 = go.Bar(
        x=['discount factor δ estimation','author estimation'],
        y=t5.filter(regex= a).iloc[7,:],
        name='Discount factor δ estimation comparision',
        textposition='auto',
        text=t5.filter(regex= a).iloc[7,:]
    )


    #fig.append_trace(trace3, 1,1)
    fig.append_trace(trace1,1,1)
    fig.append_trace(trace2,1,2)
    fig.append_trace(trace4,2,1)
    fig.append_trace(trace3,2,2)
    fig.append_trace(trace5,3,1)



    # Change the bar mode
    fig.update_layout(barmode='group',
        title='Comparison of' + " "+ f'{a}'+" "+ 'function parameters between my result and the result of authors')
    return fig.show()

In [234]:
plot_func0("exp")

In [39]:
#Just Tring code:
stvale_spec = [0.003, 0.13, 1.16, 0.75, 5e-6]
def know_how():
    def treatment_a(effort,k_scaler,s_scaler,type,st_values):
        '''
        effort: How many times the participants press the button
        
        k_scaler: This is a tool to prvent from very large number show up in the calculation process, and it would not affect on the result. The k_scaler for
                    "exp" and "power" can be different.
        s_scaler:  This is a tool to prvent from very large number show up in the calculation process, and it would not affect on the result. The s_scaler for
                    "exp" and "power" can be different.
        
        type: type can be "exp" or "power", to decide which cost function we are going to apply.
        
        st_values: This is the initial guess for the parameters k,gamma,s.

        '''
        def noweight(xdata, g, k, s, alpha, a, gift, beta, delta):
            '''
            xdata: This the vector containing the explanatory variables
            
            g, k, s: These are the same parameters from before
            
            alpha: This the pure altruism coefficient
            
            a: This is the warm glow coefficient
            
            gift: This is the gift exchange coefficient Δs

            beta: This is the present bias paramater

            delta: This is the (weekly) discount factor
            
            '''
            pay100 = xdata[0]
            gd = xdata[1] #gd is gift dummy
            dd = xdata[2] #dd is delay dummy
            dw = xdata[3] #dw is delay weeks
            paychar = xdata[4] #paychar is pay in charity treatment
            dc = xdata[5] #dc is dummy charity
            if type == "exp":
                check1 = k/k_scaler
                check2 = s/s_scaler + gift*0.4*gd + (beta**dd)*(delta**dw)*pay100 + alpha*paychar +a*0.01*dc
            elif type == "power":
                check1= max(k/k_scaler, 1e-115)
                check2= np.maximum(s/s_scaler + gift*0.4*gd + (beta**dd)*(delta**dw)*pay100 + alpha*paychar + a*0.01*dc, 1e-10)  
            f_x = (-1/g * np.log(check1) + 1/g*np.log(check2))
            return f_x

    # Find the solution to the problem by non-linear least squares 
        
        st_valuesnoweight = np.concatenate((st_values,stvale_spec)) # starting values

        args = [df.loc[df['samplenw']==1].payoff_per_100, df.loc[df['samplenw']==1].gift_dummy, df.loc[df['samplenw']==1].delay_dummy,
                df.loc[df['samplenw']==1].delay_wks, df.loc[df['samplenw']==1].payoff_charity_per_100, df.loc[df['samplenw']==1].dummy_charity]

        sol = opt.curve_fit(noweight, 
                        args,
                        df.loc[df['samplenw']==1,effort],
                        st_valuesnoweight)
        
        sol_result = [0]*8
        se_result = [0]*8
        se = np.sqrt(np.diagonal(sol[1]))
        for i in range(0,len(sol[0])):
            if i==5:
                sol_result[i] = '{0:.2e}'.format(Decimal(sol[0][i]))
                se_result[i] = '{0:.2e}'.format(Decimal(se[i]))   
            else:
                sol_result[i] = round(sol[0][i],3)
                se_result[i] = round(se[i],3)
        if type == "power":
            nwest_aut = [20.546, 5.12e-70, 3.17e-06, 0.0064462, 0.1818249, 0.0000204, 1.357934, 0.7494928]
            sse_our = np.sum((noweight(args,*sol[0])-df.loc[df['samplenw']==1,effort])**2)
            sse_aut = np.sum((noweight(args,*nwest_aut)-df.loc[df['samplenw']==1,effort])**2)
            #return sol[0],np.sqrt(np.diagonal(sol[1])),sse_our,sse_aut
            return sol_result,se_result,sse_our,sse_aut
        elif type == "exp":
            #return sol[0],np.sqrt(np.diagonal(sol[1]))
            return sol_result,se_result
    be56,se56 = treatment_a("buttonpresses_nearest_100",k_scaler = 1e+16, s_scaler= 1e+6,type = "exp",st_values = st_values_exp)
    bp53,se53,sse_our,sse_aut  = treatment_a("logbuttonpresses_nearest_100",k_scaler = 1e+57, s_scaler= 1e+6,type = "power",st_values = st_values_power)
    return be56,se56,bp53,se53,sse_our,sse_aut

be56,se56,bp53,se53,sse_our,sse_aut = know_how()
bp53
    


C:\Users\ua896\AppData\Local\Temp\ipykernel_50424\189192568.py:47: RuntimeWarning: invalid value encountered in log
  f_x = (-1/g * np.log(check1) + 1/g*np.log(check2))


[21.082, 0.0, 1.452, 0.013, 0.265, '3.25e-5', 1.616, 0.751]

In [40]:
print('The sum of squared errors using our estimates is: ' + str(sse))
print("The sum of squared errors using the authors'estimates is: " + str(sse_aut))
# The difference of the result between original code and here is also becuase the scale is different. Here follow the scale of paper.
# If don't change back the scale from scaler, the result is exactly the same as the one in original python code.

The sum of squared errors using our estimates is: 670.61
The sum of squared errors using the authors'estimates is: 115020.3364547974


In [41]:
# Create the sample used for Table 6 panel A
df['samplepr'] = (df['treatment'].isin(['1.1','1.2','1.3','6.1','6.2'])).astype(int)

In [42]:
#Define the inital guess for prob_weight and curvature
prob_weight_init = [0.2]
curv_init = [0.5]

In [43]:
def treatment_prob(effort,k_scaler,s_scaler,type,st_values,curve):
    '''
    effort: How many times the participants press the button
    
    k_scaler: This is a tool to prvent from very large number show up in the calculation process, and it would not affect on the result. The k_scaler for
                   "exp" and "power" can be different.
    s_scaler:  This is a tool to prvent from very large number show up in the calculation process, and it would not affect on the result. The s_scaler for
                   "exp" and "power" can be different.
    
    type: type can be "exp" or "power", to decide which cost function we are going to apply.
    
    st_values: This is the initial guess for the parameters k,gamma,s.

    curve: the curvature of the value function

    '''
    
    def probweight(xdata, g, k, s, p_weight): 
        '''
        xdata: This the vector containing the explanatory variables
           
        g, k, s: These are the same parameters from before
        
        p_weight: p_weight is the probability weighting coefficient 
          
        '''     
        pay100 = xdata[0]
        wd = xdata[1]
        prob = xdata[2]
    
        if type == "exp":
            check1 = k/k_scaler
            check2 = s/s_scaler+ p_weight**wd*prob*pay100**curve
        else :
            check1 = max(k/k_scaler, 1e-115)
            check2 = np.maximum(s/s_scaler + p_weight**wd*prob*pay100**curve, 1e-10)
        
        f_x = (-1/g * np.log(check1) + 1/g*np.log(check2))
        return f_x
    
    st_valuesprobweight = np.concatenate((st_values,prob_weight_init))
    args = [df.loc[df['samplepr']==1].payoff_per_100, df.loc[df['samplepr']==1].weight_dummy, df.loc[df['samplepr']==1].prob]
    sol = opt.curve_fit(probweight,
                    args,
                    df.loc[df['samplepr']==1,effort],
                    st_valuesprobweight)
    
    #Transform to easily understanding and comparable format
    solo = [i/j for i,j in zip(sol[0],[1,k_scaler,s_scaler,1,1])]
    se = [i/j for i,j in zip(np.sqrt(np.diagonal(sol[1])),[1,k_scaler,s_scaler,1,1])]
    sol_result = [0]*4
    se_result = [0]*4
    for i in range(0,len(solo)):
        if i==1 or i==2:
            sol_result[i] = '{0:.2e}'.format(Decimal(solo[i]))
            se_result[i] = '{0:.2e}'.format(Decimal(se[i]))
        else:
            sol_result[i] = round(solo[i],4)
            se_result[i] = round(se[i],4)        
   
    return np.append(sol_result,curve),np.append(se_result,0) 


In [44]:
be64,se64= treatment_prob("buttonpresses_nearest_100",k_scaler = 1e+16, s_scaler= 1e+6,type = "exp",st_values = st_values_exp,curve =1)
be65,se65 = treatment_prob("buttonpresses_nearest_100",k_scaler = 1e+16, s_scaler= 1e+6,type = "exp",st_values = st_values_exp,curve =0.88)
bp61,sp61 = treatment_prob("logbuttonpresses_nearest_100",k_scaler = 1e+57, s_scaler= 1e+6,type = "power",st_values = st_values_power,curve =1)
bp62,sp62 = treatment_prob("logbuttonpresses_nearest_100",k_scaler = 1e+57, s_scaler= 1e+6,type = "power",st_values = st_values_power,curve =0.88)
be64

array(['0.0134', '2.42e-14', '1.64e-5', '0.239', '1'], dtype='<U32')

In [190]:
def treatment_prob_curve(effort,k_scaler,s_scaler,type,st_values):
    '''
    effort: How many times the participants press the button
    
    k_scaler: This is a tool to prvent from very large number show up in the calculation process, and it would not affect on the result. The k_scaler for
                   "exp" and "power" can be different.
    s_scaler:  This is a tool to prvent from very large number show up in the calculation process, and it would not affect on the result. The s_scaler for
                   "exp" and "power" can be different.
    
    type: type can be "exp" or "power", to decide which cost function we are going to apply.
    
    st_values: This is the initial guess for the parameters k,gamma,s.


    '''
    def probweight(xdata, g, k, s, p_weight,curve):    
        '''
        xdata: This the vector containing the explanatory variables
           
        g, k, s: These are the same parameters from before
        
        p_weight: p_weight is the probability weighting coefficient 

        curve: the curvature of the value function
          
        '''   
        pay100 = xdata[0]
        wd = xdata[1]
        prob = xdata[2]
    
        if type == "exp":
            check1 = k/k_scaler
            check2 = s/s_scaler+ p_weight**wd*prob*pay100**curve
        else :
            check1 = max(k/k_scaler, 1e-115)
            check2 = np.maximum(s/s_scaler + p_weight**wd*prob*pay100**curve, 1e-10)
        
        f_x = (-1/g * np.log(check1) + 1/g*np.log(check2))
        return f_x
    
    st_valuesprobweight = np.concatenate((st_values,prob_weight_init,curv_init))
    #st_valuesprobweight = np.concatenate((st_valuesprobweight,curv_init))
    #args = [df.loc[df['samplepr']==1].payoff_per_100, df.loc[df['samplepr']==1].weight_dummy, df.loc[df['samplepr']==1].prob]
    args = [df.loc[df['samplepr']==1].payoff_per_100, df.loc[df['samplepr']==1].weight_dummy, df.loc[df['samplepr']==1].prob]
    sol = opt.curve_fit(probweight,
                    args,
                    df.loc[df['samplepr']==1,effort],
                    st_valuesprobweight)
    solo = [i/j for i,j in zip(sol[0],[1,k_scaler,s_scaler,1,1])]
    se = [i/j for i,j in zip(np.sqrt(np.diagonal(sol[1])),[1,k_scaler,s_scaler,1,1])]
    sol_result = [0]*5
    se_result = [0]*5
    for i in range(0,len(solo)):
        if i==1 or i==2:
            sol_result[i] = '{0:.2e}'.format(Decimal(solo[i]))
            se_result[i] = '{0:.2e}'.format(Decimal(se[i]))
        else:
            sol_result[i] = round(solo[i],4)
            se_result[i] = round(se[i],4)   
    return sol_result,se_result

In [186]:
#The estimation result
be66, se66 = treatment_prob_curve("buttonpresses_nearest_100",k_scaler = 1e+16, s_scaler= 1e+6,type = "exp",st_values = st_values_exp)
bp63, sp63 = treatment_prob_curve("logbuttonpresses_nearest_100",k_scaler = 1e+57, s_scaler= 1e+6,type = "power",st_values = st_values_power)
be66

[0.01, '5.46e-8', '3.14e-3', 4.3, 0.47]

In [187]:
#author result#This part is not in the original code
p_est_autp1 = [round(20.59,2),'{0:.2e}'.format(Decimal(3.77e-70)),'{0:.2e}'.format(Decimal(2.66e-06)),round(0.19,2),round(1,0)]
p_est_autp2 = [round(18.87,2),'{0:.2e}'.format(Decimal(3.92e-64)),'{0:.2e}'.format(Decimal(6.22e-06)),round(0.38,2),round(0.88,2)]
p_est_autp3 = [round(19.64,2),'{0:.2e}'.format(Decimal(1.02e-66)),'{0:.2e}'.format(Decimal(3.75e-06)),round(0.30,2),round(0.92,2)]
###
e_est_autp1 = [round(0.0134,2),'{0:.2e}'.format(Decimal(2.42e-14)),'{0:.2e}'.format(Decimal(1.65e-5)),round(0.24,2),round(1,0)]
e_est_autp2 = [round(0.0119,2),'{0:.2e}'.format(Decimal(7.5e-13)),'{0:.2e}'.format(Decimal(5.55e-5)),round(0.47,2),round(0.88,2)]
e_est_autp3 = [round(0.0072,2),'{0:.2e}'.format(Decimal(5.46e-08)),'{0:.2e}'.format(Decimal(3.14e-3)),round(4.3,2),round(0.47,2)]

In [188]:
# Create the dataframe relative to table 6
# In this table, author's result also be added.#改
pnames = ["Curvature γ of cost function", "Level k of cost of effort", "Intrinsic motivation s", "Probability weighting π (1%) (in %)",
          "Curvature of utility over piece rate"]

t6 = pd.DataFrame({'parameters':pnames,'p_est1':bp61,'p_se1':sp61,'p_aut1':p_est_autp1,
                   'p_est2':bp62,'p_se2':sp62,'p_aut2':p_est_autp2,
                   'p_est3':bp63,'p_se3':sp63,'p_aut3':p_est_autp3,
                   'e_est4':be64,'e_se4':se64,"e_aut1":e_est_autp1,
                   'e_est5':be65,'e_se5':se65,"e_aut2":e_est_autp2,
                   'e_est6':be66,'e_se6':se66,"e_aut3":e_est_autp3})
display(t6)

,parameters,p_est1,p_se1,p_aut1,p_est2,p_se2,p_aut2,p_est3,p_se3,p_aut3,e_est4,e_se4,e_aut1,e_est5,e_se5,e_aut2,e_est6,e_se6,e_aut3
0,Curvature γ of cost function,20.9475,5.7752,20.59,18.9623,5.2711,18.87,19.64,17.32,19.64,0.0134,0.0026,0.01,0.0119,0.0023,0.01,0.01,0.0,0.01
1,Level k of cost of effort,3.89e-71,1.70e-69,3.77e-70,1.96e-64,7.68e-63,3.92e-64,1.01e-66,1.36e-64,1.02e-66,2.42e-14,1.29e-13,2.42e-14,7.50e-13,3.56e-12,7.50e-13,5.46e-8,3.70e-7,5.46e-8
2,Intrinsic motivation s,1.57e-6,4.23e-6,2.66e-6,5.96e-6,1.47e-5,6.22e-6,3.75e-6,4.17e-5,3.75e-6,1.64e-5,2.40e-5,1.65e-5,5.55e-5,7.20e-5,5.55e-5,3.14e-3,7.50e-3,3.14e-3
3,Probability weighting π (1%) (in %),0.1928,0.173,0.19,0.3732,0.3042,0.38,0.3,1.57,0.3,0.239,0.1427,0.24,0.466,0.2472,0.47,4.3,5.46,4.3
4,Curvature of utility over piece rate,1,0,1,0.88,0,0.88,0.92,0.93,0.92,1,0,1,0.88,0,0.88,0.47,0.24,0.47


In [138]:
t6.filter(regex= r'p_est').iloc[1,0:1]

p_est1    3.89e-71
Name: 1, dtype: object

In [168]:
#df2 = t6.filter(regex = r'(p_|para)',axis = 1)
#df2 = df2.T
#df2 = df2.reset_index
#t6 = t6.iloc[:,1:17].astype(float)
#t6.filter(regex= r'p_est').iloc[0,:]

import plotly.graph_objects as go
func=['power_func_with_curv=1', 'power_func_with_curv=0.88', 'power_func_3_with_optimal_curv','power_func_3_with_optimal_curv']

fig = go.Figure(data=[
    go.Bar(name='Curvature γ of cost function', x=func, y=t6.filter(regex= r'e_est').iloc[3,:],textposition='auto',text=t6.filter(regex= r'e_est').iloc[3,:]),
    #go.Bar(name='Result of author', x=func, y=t6.filter(regex= r'p_aut').iloc[0,:],text=t6.filter(regex= r'p_aut').iloc[0,:],textposition='auto')
])
# Change the bar mode
fig.update_layout(barmode='group',yaxis_title='estimation_gamma',
    title='Comparison between our result and the result of authors',)
fig.show()
#


In [181]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
func=['with_curv=1', 'with_curv=0.88', 'with_optimal_curv']

fig = make_subplots(rows = 1, cols= 2, shared_yaxes=False,
                    subplot_titles=('Estimation of gamma', 'Estimation of Probability weighting'))

trace1 = go.Bar(
    x=func,
    y=t6.filter(regex= 'p_est').iloc[0,:],
    name='my gamma estimation',
    textposition='auto',
    text=t6.filter(regex= 'p_est').iloc[0,:]
)

trace2 = go.Bar(
    x=func,
    y=t6.filter(regex= 'p_aut').iloc[0,:],
    name='result of author',
    textposition='auto',
    text=t6.filter(regex= 'p_aut').iloc[0,:]
)

trace3 = go.Bar(
    x=func,
    y=t6.filter(regex= 'p_aut').iloc[3,:],
    name='result of author',
    textposition='auto',
    text=t6.filter(regex= 'p_aut').iloc[3,:]
)

trace4 = go.Bar(
    x=func,
    y=t6.filter(regex= 'p_est').iloc[3,:],
    name='my probability weight estimation',
    textposition='auto',
    text=t6.filter(regex= 'p_est').iloc[3,:]
)


#fig.append_trace(trace3, 1,1)
fig.append_trace(trace1,1,1)
fig.append_trace(trace2,1,1)
fig.append_trace(trace4,1,2)
fig.append_trace(trace3,1,2)



# Change the bar mode
fig.update_layout(barmode='group',
    title='Comparison of power function parameters between my result and the result of authors')
fig.show()
#iplot(fig)
#yaxis_title='estimation_gamma'

In [195]:
func=['with_curv=1', 'with_curv=0.88', 'with_optimal_curv']
fig = make_subplots(rows = 1, cols= 2, shared_yaxes=False,
                    subplot_titles=('Estimation of gamma', 'Estimation of Probability weighting'))

def plot_func(type1,type2,type3):
    fig = make_subplots(rows = 1, cols= 2, shared_yaxes=False,
                    subplot_titles=('Estimation of gamma', 'Estimation of Probability weighting'))
    trace1 = go.Bar(
        x=func,
        y=t6.filter(regex= f'{type1}').iloc[0,:],
        name='my gamma estimation',
        textposition='auto',
        text=t6.filter(regex= f'{type1}').iloc[0,:]
    )

    trace2 = go.Bar(
        x=func,
        y=t6.filter(regex= f'{type2}').iloc[0,:],
        name='result of author',
        textposition='auto',
        text=t6.filter(regex= f'{type2}').iloc[0,:]
    )

    trace3 = go.Bar(
        x=func,
        y=t6.filter(regex= f'{type2}').iloc[3,:],
        name='result of author',
        textposition='auto',
        text=t6.filter(regex= f'{type2}').iloc[3,:]
    )

    trace4 = go.Bar(
        x=func,
        y=t6.filter(regex= f'{type1}').iloc[3,:],
        name='my probability weight estimation',
        textposition='auto',
        text=t6.filter(regex= f'{type1}').iloc[3,:]
    )
    fig.append_trace(trace1,1,1)
    fig.append_trace(trace2,1,1)
    fig.append_trace(trace4,1,2)
    fig.append_trace(trace3,1,2)
    fig.update_layout(barmode='group',
    title="Comparison of"+" " +f'{type3}' +" "+ 'function parameters between my result and the result of authors')
    return fig.show()

plot_func("e_est","e_aut","exponential")

    


In [131]:
new_df = t6.filter(regex=r"(parameters|p_est)",axis = 1)
new_df = new_df.T
new_df = new_df.reset_index()
new_df = new_df.rename(columns=new_df.iloc[0,:])
new_df = new_df.iloc[1:4,:]
new_dff = new_df.iloc[:,1:6]
new_dff = new_dff.astype(float)
new_dff['parameters'] = new_df['parameters']
new_dff.dtypes
#new_df.rename(columns={new_df.iloc[:,1]})

Curvature γ of cost function            float64
Level k of cost of effort               float64
Intrinsic motivation s                  float64
Probability weighting π (1%) (in %)     float64
Curvature of utility over piece rate    float64
parameters                               object
dtype: object

In [146]:
import plotly.express as px
#new_dff = new_dff.sort_values(by="Curvature γ of cost function")
fig = px.bar(new_dff, x="parameters",y = "Level k of cost of effort",color = "Curvature of utility over piece rate",
             width=500,range_y=[1e-71,1e-62])
fig.show()


In [52]:
import plotly.express as px
new_dff = new_dff.sort_values(by="Probability weighting π (1%) (in %)")


fig = px.bar(new_dff, x="index",y = "Intrinsic motivation s",color = 'index',
             width=500)#color_discrete_map={"p_est1":"red","p_est2	":"green","p_est3":"blue"}
#fig.update_layout(yaxis=dict(autorange="reversed"))
fig.update_layout(yaxis_range=[1e-7, 9e-6])
fig.show()


ValueError: Value of 'x' is not the name of a column in 'data_frame'. Expected one of ['Curvature γ of cost function', 'Level k of cost of effort', 'Intrinsic motivation s', 'Probability weighting π (1%) (in %)', 'Curvature of utility over piece rate', 'parameters'] but received: index
 To use the index, pass it in directly as `df.index`.

In [ ]:
#####################################################################333

In [ ]:
df_wrong = pd.read_csv('D:\\Temp\\epp_final\\bld\\python\\data\\data_modify.csv')
df_wrong.dtypes


buttonpresses                   float64
treatment                       float64
treatmentname                    object
payoff_per_100                  float64
payoff_charity_per_100          float64
dummy_charity                     int64
delay_wks                         int64
delay_dummy                       int64
gift_dummy                        int64
prob                            float64
weight_dummy                      int64
buttonpresses_nearest_100       float64
logbuttonpresses_nearest_100    float64
dummy1                            int64
samplenw                          int64
samplepr                          int64
dtype: object